# Notebook to Perform Pytorch Profiler

# Setup

In [1]:
!pip install -U peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: peft
    Found existing installation: peft 0.13.2
    Uninstalling peft-0.13.2:
      Successfully uninstalled peft-0.13.2


In [2]:
!pip install torch transformers torch_pruning evaluate peft accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import evaluate
import accelerate

# For Pruning
import torch.nn.utils.prune as prune
import torch_pruning as tp

# For Quantization and LoRA
from transformers import BertForSequenceClassification, AutoTokenizer, BitsAndBytesConfig, AutoModelForCausalLM, AutoModelForSequenceClassification, AutoConfig, AutoModel
from transformers import Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

import pandas as pd
import numpy as np
import kagglehub

import matplotlib.pyplot as plt
import seaborn as sns
import os
import random

import argparse
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Unzip Model Files

In [4]:
!cd /content/fine_tuned_optimized_bert
!unzip fine_tuned_optimized_bert.zip

Archive:  fine_tuned_optimized_bert.zip
Written using ZipTricks 5.6.0
  inflating: adapter_config.json     
  inflating: adapter_model.safetensors  
   creating: checkpoint-1000/
  inflating: checkpoint-1000/adapter_config.json  
  inflating: checkpoint-1000/adapter_model.safetensors  
  inflating: checkpoint-1000/optimizer.pt  
  inflating: checkpoint-1000/rng_state.pth  
  inflating: checkpoint-1000/scheduler.pt  
  inflating: checkpoint-1000/special_tokens_map.json  
  inflating: checkpoint-1000/tokenizer.json  
  inflating: checkpoint-1000/tokenizer_config.json  
  inflating: checkpoint-1000/trainer_state.json  
  inflating: checkpoint-1000/training_args.bin  
  inflating: checkpoint-1000/vocab.txt  
   creating: checkpoint-1500/
  inflating: checkpoint-1500/adapter_config.json  
  inflating: checkpoint-1500/adapter_model.safetensors  
  inflating: checkpoint-1500/optimizer.pt  
  inflating: checkpoint-1500/rng_state.pth  
  inflating: checkpoint-1500/scheduler.pt  
  inflating: ch

In [5]:
!cd /content/fine_tuned_optimized_gpt2
!unzip fine_tuned_optimized_gpt2.zip

Archive:  fine_tuned_optimized_gpt2.zip
Written using ZipTricks 5.6.0
  inflating: adapter_config.json     
  inflating: adapter_model.safetensors  
   creating: checkpoint-1062/
  inflating: checkpoint-1062/adapter_config.json  
  inflating: checkpoint-1062/adapter_model.safetensors  
  inflating: checkpoint-1062/merges.txt  
  inflating: checkpoint-1062/optimizer.pt  
  inflating: checkpoint-1062/rng_state_0.pth  
  inflating: checkpoint-1062/rng_state_1.pth  
  inflating: checkpoint-1062/rng_state_2.pth  
  inflating: checkpoint-1062/rng_state_3.pth  
  inflating: checkpoint-1062/scheduler.pt  
  inflating: checkpoint-1062/special_tokens_map.json  
  inflating: checkpoint-1062/tokenizer.json  
  inflating: checkpoint-1062/tokenizer_config.json  
  inflating: checkpoint-1062/trainer_state.json  
  inflating: checkpoint-1062/training_args.bin  
  inflating: checkpoint-1062/vocab.json  
   creating: checkpoint-1593/
  inflating: checkpoint-1593/adapter_config.json  
  inflating: checkp

In [6]:
!cd /content/fine_tuned_optimized_llama
!unzip fine_tuned_optimized_llama.zip

Archive:  fine_tuned_optimized_llama.zip
Written using ZipTricks 5.6.0
  inflating: README.md               
  inflating: adapter_config.json     
  inflating: adapter_model.safetensors  
   creating: checkpoint-1062/
  inflating: checkpoint-1062/README.md  
  inflating: checkpoint-1062/adapter_config.json  
  inflating: checkpoint-1062/adapter_model.safetensors  
  inflating: checkpoint-1062/optimizer.pt  
  inflating: checkpoint-1062/rng_state_0.pth  
  inflating: checkpoint-1062/rng_state_1.pth  
  inflating: checkpoint-1062/rng_state_2.pth  
  inflating: checkpoint-1062/rng_state_3.pth  
  inflating: checkpoint-1062/scheduler.pt  
  inflating: checkpoint-1062/special_tokens_map.json  
  inflating: checkpoint-1062/tokenizer.json  
  inflating: checkpoint-1062/tokenizer_config.json  
  inflating: checkpoint-1062/trainer_state.json  
  inflating: checkpoint-1062/training_args.bin  
   creating: checkpoint-1593/
  inflating: checkpoint-1593/README.md  
  inflating: checkpoint-1593/adap

In [ ]:
os.environ['HF_TOKEN'] = ""

# Review Text

Get review straight from amazon

In [8]:
positive_review = "The Apple Watch Series 10 (GPS 42mm case) is an outstanding smartwatch that seamlessly blends style and functionality. Its sleek design and vibrant display make it a pleasure to wear, while the robust health and fitness tracking features provide valuable insights into your daily activity. With improved battery life and a range of customizable watch faces, it caters to both casual users and fitness enthusiasts alike. The integration with the Apple ecosystem enhances its usability, allowing for quick access to notifications and apps. Overall, the Apple Watch Series 10 is a fantastic choice for anyone looking to elevate their smartwatch experience!"

In [9]:
neutral_review = "I got this instead of a fitbit to stay connected through my devices (all apple), but honestly my feelings are mixed. This is my first time having an apple watch so I might not know something but the battery life just kills the mood, i can’t go a full day or day and a half without charging this thing. Idk about anything else but that just doesn’t sit right with me, is very tedious and i can’t even track that much of my sleep since i have to charge it every night, besides from that, it works pretty well, I like being able to do stuff from here instead of using my phone. I’ll probably buy a new fitbit model instead, nothing wrong with how it works ig just u know…the battery "

In [10]:
negative_review = "This was a refurnished product, but when I received it, it had an Apple activation lock on it because the previous owner had used it and not signed out of their iCloud account. I contacted Apple to try and get it removed without any success and Amazon told me that they could not help me either even though it was an Amazon purchase. I ended up buying a new one cheaper a few weeks later after I couldn’t get help and returned the refurbished product back to Amazon."

In [11]:
All_reviews = [positive_review, neutral_review, negative_review]

# Test Optimized BERT

Load Optimized Bert

In [12]:
_MODEL_NAME = "bert"
model_path = "/content/fine_tuned_optimized_bert"

## Evaluate

Pass review to model, and run pytorch profiler to save results

In [13]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftConfig, PeftModel

In [14]:
from torch.profiler import profile, record_function, ProfilerActivity

In [15]:
peft_model_id = model_path

config = AutoConfig.from_pretrained(model_path)

model = AutoModelForSequenceClassification.from_pretrained(
    model_path,
    num_labels=3,
    use_auth_token=True,
    ignore_mismatched_sizes=True
)

model.config.pad_token_id = model.config.eos_token_id
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [16]:
# Ensure model is in evaluation mode
model.eval()

for amazon_review in All_reviews:
  # Tokenize the input text
  inputs = tokenizer(amazon_review, return_tensors="pt", truncation=True, padding=True, max_length=512)

  # Perform inference
  with torch.no_grad():
      outputs = model(**inputs)

  # Get predicted sentiment
  logits = outputs.logits
  predicted_class = torch.argmax(logits, dim=1).item()

  # Map the predicted class to sentiment labels
  sentiment_labels = {0: "negative", 1: "neutral", 2: "positive"}
  predicted_sentiment = sentiment_labels[predicted_class]

  # Print the result
  print(f"Sentiment: {predicted_sentiment}")

Sentiment: positive
Sentiment: positive
Sentiment: negative


# Test Optimized GPT2

Load Optimized GPT2

In [17]:
_MODEL_NAME = "gpt2"
model_path = "/content/fine_tuned_optimized_gpt2"

In [18]:
peft_model_id = model_path

config = AutoConfig.from_pretrained(model_path)


#Load the base model
model = AutoModelForSequenceClassification.from_pretrained(
    model_path,
    num_labels=3,
    use_auth_token=True,
    ignore_mismatched_sizes=True
)

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

## Evaluate

Pass review to model, and run pytorch profiler to save results

In [19]:
# Ensure model is in evaluation mode
model.eval()

for amazon_review in All_reviews:
  # Tokenize the input text
  inputs = tokenizer(amazon_review, return_tensors="pt", truncation=True, padding=True, max_length=512)

  # Perform inference
  with torch.no_grad():
      outputs = model(**inputs)

  # Get predicted sentiment
  logits = outputs.logits
  predicted_class = torch.argmax(logits, dim=1).item()

  # Map the predicted class to sentiment labels
  sentiment_labels = {0: "negative", 1: "neutral", 2: "positive"}
  predicted_sentiment = sentiment_labels[predicted_class]

  # Print the result
  print(f"Sentiment: {predicted_sentiment}")

Sentiment: positive
Sentiment: negative
Sentiment: negative


# Test Optimized LLAMA

Load Optimized Llama

In [20]:
_MODEL_NAME = "llama"
model_path = "/content/fine_tuned_optimized_llama"

In [21]:
peft_model_id = model_path

config = AutoConfig.from_pretrained(model_path)

#Load the base model
model = AutoModelForSequenceClassification.from_pretrained(
    model_path,
    num_labels=3,
    use_auth_token=True,
    ignore_mismatched_sizes=True
)

model.config.pad_token_id = model.config.eos_token_id
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

## Evaluate

Pass review to model, and run pytorch profiler to save results

In [22]:
# Ensure model is in evaluation mode
model.eval()

for amazon_review in All_reviews:
  # Tokenize the input text
  inputs = tokenizer(amazon_review, return_tensors="pt", truncation=True, padding=True, max_length=512)

  # Perform inference
  with torch.no_grad():
      outputs = model(**inputs)

  # Get predicted sentiment
  logits = outputs.logits
  predicted_class = torch.argmax(logits, dim=1).item()

  # Map the predicted class to sentiment labels
  sentiment_labels = {0: "negative", 1: "neutral", 2: "positive"}
  predicted_sentiment = sentiment_labels[predicted_class]

  # Print the result
  print(f"Sentiment: {predicted_sentiment}")

Sentiment: positive
Sentiment: neutral
Sentiment: negative
